In [ ]:
import geemap
import ee
ee.Initialize()
import scripts.analysis_functions as af
import scripts.fire_info as fi
import scripts.get_image_collections as gic
import scripts.utils as utils

# For yearly fires (takes a FeatureCollection)

In [ ]:
yr='2021'
fires = ee.FeatureCollection(f"projects/pyregence-ee/assets/conus/nifc/nifc_fires_{yr}_gte500acres_20221004")
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')

# fires = ee.FeatureCollection("users/kwoodward/burn-severity/perimeters/nifc_fires_2020_20211220")

acre_min = 0 # can filter by Acres field 
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')

# fires_w_filters = gic.remove_recent(fires).filter(ee.Filter.gte('Acres', acre_min))

# print(f'Removing very recent fires... removing fires less than {acre_min} acres')
# print(f'Computing Burn Severity for {fires_w_filters.size().getInfo()} fires')

latest_fire = fires.sort('Discovery',False).first()
# latest_fire = fires_w_filters.sort('Discovery',False).first()
print('latest fire:\n', latest_fire.getInfo()['properties'])

run_mode = fi.config_mode(latest_fire)
print(f'run mode: {run_mode}')

def bs_calc(feat: ee.Feature):
    fire = ee.Feature(feat)
    # pass one fire feature into new fire_info function
    region,pre_start,pre_end,fire_start,fire_end = fi.get_fire_info_from_feature(fire, run_mode)

    # key consideration for the recent mode is how to handle more recent fires that may have cloudy imagery... 
    # can put date filter value in gic.remove_recent() further back from today's date (currently its 30 days) so all fires left in the FeatureColl should have enough images to 
    # get a good .mean() composite using gic.make_pre_composite(), or keep the date filter pretty close to today's date and let it pull T1 realtime and toa imgs with no cloudBusting (as is done currently) with gic.make_nrt_composite()
    # and just caveat that the more recent fire perims will have less reliable BS metric; its easy to run this every few months to get a better severity estimate on those more recent fires
    sensor = "landsat"
    pre_collection = gic.get_image_collection(sensor,region,pre_start,pre_end,landsatCol='bestls',cloudBustingMethod='bust')
    pre_img = gic.get_composite(pre_collection,gic.make_pre_composite,pre_start,pre_end)

    if run_mode == 'historical':
        post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end,landsatCol='bestls', cloudBustingMethod='bust')
        post_img = gic.get_composite(post_collection,gic.make_pre_composite,fire_start, fire_end) 

    else: # recent
        post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end)
        post_img = gic.get_composite(post_collection,gic.make_nrt_composite, sensor) 
    
    rdnbr = af.rdnbr(pre_img,post_img)
    miller = af.miller_thresholds(rdnbr)
    
    return ee.Image(miller).clip(region).select('MillersThresholds').toByte()

bs_imgColl = ee.FeatureCollection(fires).map(bs_calc)
# bs_imgColl = ee.FeatureCollection(fires_w_filters).map(bs_calc)
annual_conus_bs = ee.ImageCollection(bs_imgColl).max().add(1).rename('SEVERITY') #shift severity values +1 so 0 can be nodata

In [3]:
Map = geemap.Map()
Map.addLayer(annual_conus_bs, {"min":1,"max":4,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'bs img')

Map.setCenter(-118.6633, 36.2576, 7)
Map

Map(center=[36.2576, -118.6633], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

# For one fire (takes a Feature)
### Select a fire from a pre-existing fire featurecollection or provide your own fire feature asset (must have 'Discovery' property with value of format ee.String('yyyy-mm-dd') )


In [4]:
#fire = fires_w_filters.filter(ee.Filter.eq('Name', 'Dixie')).sort('Acres',False).first()
fire = fires_w_filters.filter(ee.Filter.eq('Name', 'Creek 5')).sort('Acres',False).first()

run_mode = fi.config_mode(fire)
print(f'run mode: {run_mode}')

print(fire.getInfo()['properties'])

# pass one fire feature into new fire_info function
region,pre_start,pre_end,fire_start,fire_end = fi.get_fire_info_from_feature(fire, run_mode)
print(pre_start.getInfo())
print(pre_end.getInfo())
print(fire_start.getInfo())
print(fire_end.getInfo())

sensor = "landsat"
pre_collection = gic.get_image_collection(sensor,region,pre_start,pre_end,landsatCol='bestls',cloudBustingMethod='bust')
pre_img = gic.get_composite(pre_collection,gic.make_pre_composite,pre_start,pre_end)
print(pre_collection.size().getInfo())
print(pre_img.bandNames().getInfo())

if run_mode == 'historical':
    post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end,landsatCol='bestls', cloudBustingMethod='bust')
    post_img = gic.get_composite(post_collection,gic.make_pre_composite,fire_start, fire_end)
else: # recent 
    post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end)
    post_img = gic.get_composite(post_collection,gic.make_nrt_composite, sensor)

print(post_collection.size().getInfo())
print(post_img.bandNames().getInfo())

rdnbr = af.rdnbr(pre_img,post_img)
miller = af.miller_thresholds(rdnbr).add(1).clip(region).select('MillersThresholds').toByte()

run mode: historical
{'Acres': 3052.16115021611, 'Discovery': '2020-12-24', 'Discovery_Date': {'type': 'Date', 'value': 1608768000000}, 'Name': 'Creek 5', 'OBJECTID': 861}
2019-9-26
2019-12-25
2021-9-25
2021-12-24
6
['ca', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'cirrus', 'temp1', 'temp2', 'BQA', 'date']
4
['ca', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'cirrus', 'temp1', 'temp2', 'BQA', 'date']


In [8]:
# look at post_img and determine for yourself if its good enough to be using for analysis 
Map = geemap.Map()
Map.addLayer(pre_img,{"bands":["swir2","nir","red"],"min":0,"max":0.848},"pre")
Map.addLayer(post_img,{"bands":["swir2","nir","red"],"min":0,"max":0.848},"post")
Map.addLayer(rdnbr.clip(region),{"min":0,"max":1500,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'rdnbr')
Map.addLayer(miller.clip(region),{"min":1,"max":4,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'miller')
Map.addLayer(region, {}, 'region')
Map.centerObject(region, 11)
Map



Map(center=[33.38055510100062, -117.28694918122679], controls=(WidgetControl(options=['position', 'transparent…

## Exports
### To export an annual set of fires, use export_type = 'conus', if its a single fire, use export_type = 'single_fire'

In [12]:
CA = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME','California')).first()
# Map.addLayer(ee.Feature(CA))
# print(ee.Feature(CA).getInfo()['properties'])
                                                      
# Single FIre export templates
# # To Google Drive
# desc = 'dixie_bs2020_test_20211227'
# utils.exportImgtoDrive(miller,
#                         desc,
#                         export_type='single_fire',
#                         folder='BurnSeverity_outputs',
#                         export=True,
#                         default=sensor)


# # To Asset
# desc = 'dixie_bs2020_test_20211227'
# utils.exportImgtoAsset(miller, 
#                         desc=desc, 
#                         asset_folder= "users/kwoodward/burn-severity/bs_images", 
#                         export_type='single_fire',
#                         export=True,
#                         default=sensor)

# # CONUS/multi-fire export template
# # To Google Drive
# desc = 'CA_bs2021_nifcFires_20220713'
# utils.exportImgtoDrive(annual_conus_bs.clip(CA),
#                         desc,
#                         export_type='conus',
#                         folder='BurnSeverity_outputs',
#                         export=False)
#                         #default=sensor)


# To Asset
desc = 'CA_bs2021_nifcFires_20220713'
utils.exportImgtoAsset(annual_conus_bs.clip(CA), 
                        desc=desc, 
                        asset_folder= "users/kwoodward/burn-severity/bs_images", 
                        export_type='conus',
                        export=True)
                        #default=sensor)

export task started: users/kwoodward/burn-severity/bs_images/CA_bs2021_nifcFires_20220713
